In [1]:
from openfermion.hamiltonians import MolecularData, load_molecular_hamiltonian
from openfermion.ops import FermionOperator
from openfermion.transforms import bravyi_kitaev
from openfermion.utils import hermitian_conjugated
from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli
from pyquil.paulis import PauliSum
from pyquil.api import WavefunctionSimulator
from scipy.optimize import minimize
from pyquil import Program
from pyquil.gates import *

import numpy as np
import functools

import matplotlib.pyplot as plt
%matplotlib inline

from openfermion.utils import uccsd_generator
from forestopenfermion import exponentiate


In [ ]:
sim = WavefunctionSimulator()
numQubit = 12

def solve_vqe_ucc_h2(hamiltonian: PauliSum, degree) -> float:
    # Construct a variational quantum eigensolver solution to find the lowest
    # eigenvalue of the given hamiltonian
    
    # degree 1: up to T_1
    # degree 2: up to T_2
    
    numParams = 0
    if degree == 1:
        numParams = 2
    elif degree == 2:
        numParams = 3
    params_init = np.random.rand(numParams)*0.5-0.25
    
    def ansatz_energy(params):
        p = None
        p = Program()
        
        # Reference state
        p += X(0)
        
        single_amp = []
        double_amp = []
        
        # occupied: 0,1 unoccupied: 2,3
        count = 0
        single_amp.append([[2,0], params[count]])
        count += 1
        single_amp.append([[3,1], params[count]])
        count += 1
        
        if degree >= 2:
            double_amp.append([[2,0,3,1], params[count]])
        
        ucc_gen = uccsd_generator(single_amp, double_amp)
        ucc_program = exponentiate(bravyi_kitaev(ucc_gen)/(-1j))
        p += ucc_program

        energy = sim.expectation(p, hamiltonian).real
        return energy

    option = {}
    option['disp']=True
    params_answer = minimize(ansatz_energy, params_init, method='L-BFGS-B', options=option).x
    #print(params_answer)
    return ansatz_energy(params_answer)